# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770843674480                   -0.53    8.0         
  2   -2.772140529948       -2.89       -1.31    1.0    107ms
  3   -2.772170122022       -4.53       -2.56    1.0    137ms
  4   -2.772170706859       -6.23       -3.52    2.0    122ms
  5   -2.772170721237       -7.84       -3.92    1.0    138ms
  6   -2.772170722856       -8.79       -4.75    1.0    110ms
  7   -2.772170723006       -9.82       -5.01    2.0    127ms
  8   -2.772170723015      -11.05       -5.75    1.0    121ms
  9   -2.772170723015      -12.29       -6.46    2.0    130ms
 10   -2.772170723015      -13.83       -7.61    1.0    143ms
 11   -2.772170723015      -14.24       -8.72    2.0    131ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770600288348                   -0.53    8.0         
  2   -2.7

1.773558059530367

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770617423478                   -0.53    8.0         
  2   -2.772046993410       -2.84       -1.30    1.0    152ms
  3   -2.772082375154       -4.45       -2.65    1.0    106ms
  4   -2.772083414597       -5.98       -3.95    2.0    166ms
  5   -2.772083417657       -8.51       -4.45    2.0    123ms
  6   -2.772083417804       -9.83       -5.35    1.0    111ms
  7   -2.772083417810      -11.19       -5.95    2.0    145ms
  8   -2.772083417811      -12.94       -6.85    1.0    116ms
  9   -2.772083417811      -13.78       -7.60    2.0    148ms
 10   -2.772083417811   +  -14.21       -7.85    2.0    137ms
 11   -2.772083417811   +  -14.45       -9.40    1.0    122ms

Polarizability via ForwardDiff:       1.7725349704777307
Polarizability via finite difference: 1.773558059530367
